In [ ]:
  # The right API to pass in a prompt (of type string) is the completions API https://docs.together.ai/reference/completions-1
  # The right API to pass in a messages (of type of list of message) is The chat completions API https://docs.together.ai/reference/chat-completions-1

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
def load_env():
    _ = load_dotenv(find_dotenv())
env_dict = os.environ
for key, value in env_dict.items():
    print(f"{key}: {value} \n")

In [4]:
import requests
import json


def llama32(messages, model_size=11):
  model = f"meta-llama/Llama-3.2-{model_size}B-Vision-Instruct-Turbo"
  url = f"{os.getenv('DLAI_TOGETHER_API_BASE', 'https://api.together.xyz')}/v1/chat/completions"
  payload = {
    "model": model,
    "max_tokens": 4096,
    "temperature": 0.0,
    "stop": ["<|eot_id|>","<|eom_id|>"],
    "messages": messages
  }

  headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": f"Bearer {os.getenv('TOGETHER_API_KEY')}"
  }
  res = json.loads(requests.request("POST", url, headers=headers, data=json.dumps(payload)).content)

  if 'error' in res:
    raise Exception(res['error'])

  return res['choices'][0]['message']['content']


def llama31(prompt_or_messages, model_size=8, temperature=0.5, raw=False, debug=False):
    model = f"meta-llama/Meta-Llama-3.1-{model_size}B-Instruct-Turbo"
    if isinstance(prompt_or_messages, str):
        prompt = prompt_or_messages
        url = f"{os.getenv('DLAI_TOGETHER_API_BASE', 'https://api.together.xyz')}/v1/completions"
        payload = {
            "model": model,
            "temperature": temperature,
            "prompt": prompt
        }
    else:
        messages = prompt_or_messages
        url = f"{os.getenv('DLAI_TOGETHER_API_BASE', 'https://api.together.xyz')}/v1/chat/completions"
        payload = {
            "model": model,
            "temperature": temperature,
            "messages": messages
        }

    if debug:
        print(payload)

    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f"Bearer {os.getenv('TOGETHER_API_KEY')}"
    }

    try:
        response = requests.post(
            url, headers=headers, data=json.dumps(payload)
        )
        response.raise_for_status()  # Raises HTTPError for bad responses
        res = response.json()
    except requests.exceptions.RequestException as e:
        raise Exception(f"Request failed: {e}")

    if 'error' in res:
        raise Exception(f"API Error: {res['error']}")

    if raw:
        return res

    if isinstance(prompt_or_messages, str):
        return res['choices'][0].get('text', '')
    else:
        return res['choices'][0].get('message', {}).get('content', '')

In [5]:
from IPython.display import Markdown, display


In [6]:
#define a class for interaction in higher level language
class Conversation:
    def __init__(self, system=""):
        self.messages = []
        if system:
            self.messages.append({"role": "system", "content": system})
    def generate(self, user_question, model=405, temp=0.2): #it seems the model is not performing much better, NO THE PROBLEM WAS 
        #THE TEMPERATURE. NOW ITS PERFORMING MUCH BETTER
        self.messages.append({"role": "user", "content":user_question})
        response = llama31(self.messages, model, temperature=temp)
        self.messages.append({"role":"assistant", "content":response})
        return response
        

In [7]:
system_message = "You are a classical dance coreographer for intermediate\
ballet students. You are responsible for designing a whole barre class.\
1) Define a sequence with appropiate transitions and detailed instructions for transitions and \
movements and offer a preparation and an ending phrase\
2) Name all positions for the body, the feet and the arms while presenting the sequence\
3) Justify your decisions" 
conv = Conversation(system_message)
dance_query = """ 
You will make a sequence with pliés. The following are your tasks
These are must haves in the sequence:
a) Don't forget to include plies in second position with one foot in relevé and plies \
in fifth position.
b) Don't forget to finish each of the position with cambrés
c) First present the sequence for one side and then turn to the other side
d) Clarify what kind of music/tempo would better fit the sequence and include approximate \
timing for each exercise
"""
response = conv.generate(dance_query)
from IPython.display import Markdown, display

display(Markdown(response))

Exception: Request failed: 401 Client Error: Unauthorized for url: https://api.together.xyz/v1/chat/completions

In [9]:
response = "**Sequence: Pliés with Relevé and Cambré**\n\n**Music:** A moderate tempo with a 4/4 time signature, ideally a classical piano piece with a steady and soothing rhythm. (e.g., Chopin's Nocturne in E-flat major, Op. 9, No. 2)\n\n**Tempo:** Moderate, around 96-100 beats per minute (BPM)\n\n**Sequence for one side (right):**\n\n1. **Pliés in First Position** (4 counts)\n\t* Feet: Heels together, toes turned out\n\t* Arms: Relaxed, with a slight curve, in first position (palms facing downwards)\n\t* Body: Engaged core, lengthened spine, and relaxed shoulders\n\t* Movement: Slowly lower the body down into a plié, keeping the back straight and the knees over the toes\n\t* Transition: Straighten the legs and rise to the starting position\n2. **Pliés in Second Position with Relevé (Right Foot)** (4 counts)\n\t* Feet: Right foot in relevé, left foot on the ground in second position\n\t* Arms: Still in first position\n\t* Body: Maintain the engaged core and lengthened spine\n\t* Movement: Lower the body down into a plié, keeping the right foot in relevé and the left foot on the ground\n\t* Transition: Straighten the legs and rise to the starting position\n\t* Cambré: As you rise, stretch the right arm up and over the head, keeping the left arm in first position (2 counts)\n3. **Pliés in Fifth Position (Right Foot in Front)** (4 counts)\n\t* Feet: Right foot in front, left foot behind in fifth position\n\t* Arms: Return to first position\n\t* Body: Maintain the engaged core and lengthened spine\n\t* Movement: Lower the body down into a plié, keeping the back straight and the knees over the toes\n\t* Transition: Straighten the legs and rise to the starting position\n\t* Cambré: As you rise, stretch the right arm up and over the head, keeping the left arm in first position (2 counts)\n\n**Turn to the other side (left):**\n\nRepeat the sequence on the left side, switching the positions of the feet and arms accordingly.\n\n**Sequence for the left side:**\n\n1. **Pliés in First Position** (4 counts)\n\t* Feet: Heels together, toes turned out\n\t* Arms: Relaxed, with a slight curve, in first position (palms facing downwards)\n\t* Body: Engaged core, lengthened spine, and relaxed shoulders\n\t* Movement: Slowly lower the body down into a plié, keeping the back straight and the knees over the toes\n\t* Transition: Straighten the legs and rise to the starting position\n2. **Pliés in Second Position with Relevé (Left Foot)** (4 counts)\n\t* Feet: Left foot in relevé, right foot on the ground in second position\n\t* Arms: Still in first position\n\t* Body: Maintain the engaged core and lengthened spine\n\t* Movement: Lower the body down into a plié, keeping the left foot in relevé and the right foot on the ground\n\t* Transition: Straighten the legs and rise to the starting position\n\t* Cambré: As you rise, stretch the left arm up and over the head, keeping the right arm in first position (2 counts)\n3. **Pliés in Fifth Position (Left Foot in Front)** (4 counts)\n\t* Feet: Left foot in front, right foot behind in fifth position\n\t* Arms: Return to first position\n\t* Body: Maintain the engaged core and lengthened spine\n\t* Movement: Lower the body down into a plié, keeping the back straight and the knees over the toes\n\t* Transition: Straighten the legs and rise to the starting position\n\t* Cambré: As you rise, stretch the left arm up and over the head, keeping the right arm in first position (2 counts)\n\n**Preparation (4 counts):**\n\nStand in first position, with feet together and arms relaxed by your sides. Take a deep breath in, engaging your core and lengthening your spine. As you exhale, slowly raise your arms to first position, keeping them relaxed and curved.\n\n**Ending Phrase (4 counts):**\n\nFinish the sequence in first position, with feet together and arms in first position. Take a deep breath in, and as you exhale, slowly lower your arms to your sides, keeping them relaxed. Take a final deep breath in, and as you exhale, bow your head slightly, keeping your core engaged and your spine lengthened.\n\n**Justification:**\n\nThis sequence is designed to focus on pliés, which are a fundamental movement in ballet. By incorporating relevé and cambré, the sequence challenges the dancers to maintain balance and control while moving through different positions. The moderate tempo and 4/4 time signature provide a steady and soothing rhythm, allowing the dancers to focus on their technique and movement quality. The sequence is structured to gradually increase in difficulty, starting with pliés in first position and progressing to more challenging movements in second and fifth positions. The cambré movements add an element of expression and musicality, allowing the dancers to connect with the music and convey emotion through their movement."
display(Markdown(response))

**Sequence: Pliés with Relevé and Cambré**

**Music:** A moderate tempo with a 4/4 time signature, ideally a classical piano piece with a steady and soothing rhythm. (e.g., Chopin's Nocturne in E-flat major, Op. 9, No. 2)

**Tempo:** Moderate, around 96-100 beats per minute (BPM)

**Sequence for one side (right):**

1. **Pliés in First Position** (4 counts)
	* Feet: Heels together, toes turned out
	* Arms: Relaxed, with a slight curve, in first position (palms facing downwards)
	* Body: Engaged core, lengthened spine, and relaxed shoulders
	* Movement: Slowly lower the body down into a plié, keeping the back straight and the knees over the toes
	* Transition: Straighten the legs and rise to the starting position
2. **Pliés in Second Position with Relevé (Right Foot)** (4 counts)
	* Feet: Right foot in relevé, left foot on the ground in second position
	* Arms: Still in first position
	* Body: Maintain the engaged core and lengthened spine
	* Movement: Lower the body down into a plié, keeping the right foot in relevé and the left foot on the ground
	* Transition: Straighten the legs and rise to the starting position
	* Cambré: As you rise, stretch the right arm up and over the head, keeping the left arm in first position (2 counts)
3. **Pliés in Fifth Position (Right Foot in Front)** (4 counts)
	* Feet: Right foot in front, left foot behind in fifth position
	* Arms: Return to first position
	* Body: Maintain the engaged core and lengthened spine
	* Movement: Lower the body down into a plié, keeping the back straight and the knees over the toes
	* Transition: Straighten the legs and rise to the starting position
	* Cambré: As you rise, stretch the right arm up and over the head, keeping the left arm in first position (2 counts)

**Turn to the other side (left):**

Repeat the sequence on the left side, switching the positions of the feet and arms accordingly.

**Sequence for the left side:**

1. **Pliés in First Position** (4 counts)
	* Feet: Heels together, toes turned out
	* Arms: Relaxed, with a slight curve, in first position (palms facing downwards)
	* Body: Engaged core, lengthened spine, and relaxed shoulders
	* Movement: Slowly lower the body down into a plié, keeping the back straight and the knees over the toes
	* Transition: Straighten the legs and rise to the starting position
2. **Pliés in Second Position with Relevé (Left Foot)** (4 counts)
	* Feet: Left foot in relevé, right foot on the ground in second position
	* Arms: Still in first position
	* Body: Maintain the engaged core and lengthened spine
	* Movement: Lower the body down into a plié, keeping the left foot in relevé and the right foot on the ground
	* Transition: Straighten the legs and rise to the starting position
	* Cambré: As you rise, stretch the left arm up and over the head, keeping the right arm in first position (2 counts)
3. **Pliés in Fifth Position (Left Foot in Front)** (4 counts)
	* Feet: Left foot in front, right foot behind in fifth position
	* Arms: Return to first position
	* Body: Maintain the engaged core and lengthened spine
	* Movement: Lower the body down into a plié, keeping the back straight and the knees over the toes
	* Transition: Straighten the legs and rise to the starting position
	* Cambré: As you rise, stretch the left arm up and over the head, keeping the right arm in first position (2 counts)

**Preparation (4 counts):**

Stand in first position, with feet together and arms relaxed by your sides. Take a deep breath in, engaging your core and lengthening your spine. As you exhale, slowly raise your arms to first position, keeping them relaxed and curved.

**Ending Phrase (4 counts):**

Finish the sequence in first position, with feet together and arms in first position. Take a deep breath in, and as you exhale, slowly lower your arms to your sides, keeping them relaxed. Take a final deep breath in, and as you exhale, bow your head slightly, keeping your core engaged and your spine lengthened.

**Justification:**

This sequence is designed to focus on pliés, which are a fundamental movement in ballet. By incorporating relevé and cambré, the sequence challenges the dancers to maintain balance and control while moving through different positions. The moderate tempo and 4/4 time signature provide a steady and soothing rhythm, allowing the dancers to focus on their technique and movement quality. The sequence is structured to gradually increase in difficulty, starting with pliés in first position and progressing to more challenging movements in second and fifth positions. The cambré movements add an element of expression and musicality, allowing the dancers to connect with the music and convey emotion through their movement.

In [ ]:
response = conv.generate("Now you will design a sequence for battement tendu.\
                         You should not focus on doing all  movements in all positions but,\
                         to make tendus to all directions\
                         These are must haves:\
                         a) Fondu \
                         b) Cou de Pied \
                         c) Développé\
                         d) Cloche ")
display(Markdown(response)) #=! print

In [12]:
response = '**Sequence: Battement Tendu with Directions and Added Movements**\n\n**Music:** A moderate tempo with a 4/4 time signature, ideally a classical piano piece with a steady and lively rhythm. (e.g., Debussy\'s "Clair de Lune")\n\n**Tempo:** Moderate, around 100-104 beats per minute (BPM)\n\n**Sequence:**\n\n**Part 1: Tendus to the Front and Sides**\n\n1. **Battement Tendu to the Front** (2 counts)\n\t* Feet: First position, with the working foot (right) brushing out to the front\n\t* Arms: Relaxed, with a slight curve, in first position (palms facing downwards)\n\t* Body: Engaged core, lengthened spine, and relaxed shoulders\n\t* Movement: Brush the working foot out to the front, keeping the knee straight and the toe pointed\n\t* Transition: Bring the working foot back to first position\n2. **Battement Tendu to the Side (Right)** (2 counts)\n\t* Feet: First position, with the working foot (right) brushing out to the right side\n\t* Arms: Still in first position\n\t* Body: Maintain the engaged core and lengthened spine\n\t* Movement: Brush the working foot out to the right side, keeping the knee straight and the toe pointed\n\t* Transition: Bring the working foot back to first position\n3. **Battement Tendu to the Side (Left)** (2 counts)\n\t* Feet: First position, with the working foot (left) brushing out to the left side\n\t* Arms: Still in first position\n\t* Body: Maintain the engaged core and lengthened spine\n\t* Movement: Brush the working foot out to the left side, keeping the knee straight and the toe pointed\n\t* Transition: Bring the working foot back to first position\n\n**Part 2: Tendus to the Back and Added Movements**\n\n1. **Battement Tendu to the Back** (2 counts)\n\t* Feet: First position, with the working foot (right) brushing out to the back\n\t* Arms: Still in first position\n\t* Body: Maintain the engaged core and lengthened spine\n\t* Movement: Brush the working foot out to the back, keeping the knee straight and the toe pointed\n\t* Transition: Bring the working foot back to first position\n2. **Fondu** (2 counts)\n\t* Feet: First position, with the working foot (right) sinking into a fondu\n\t* Arms: Still in first position\n\t* Body: Maintain the engaged core and lengthened spine\n\t* Movement: Slowly lower the working foot into a fondu, keeping the back straight and the knee bent\n\t* Transition: Straighten the working leg and rise to the starting position\n3. **Cou de Pied** (2 counts)\n\t* Feet: First position, with the working foot (right) drawing to the supporting leg\n\t* Arms: Still in first position\n\t* Body: Maintain the engaged core and lengthened spine\n\t* Movement: Draw the working foot to the supporting leg, keeping the knee bent and the toe pointed\n\t* Transition: Release the working foot and bring it back to first position\n4. **Développé** (4 counts)\n\t* Feet: First position, with the working foot (right) unfolding into a développé\n\t* Arms: Still in first position\n\t* Body: Maintain the engaged core and lengthened spine\n\t* Movement: Slowly unfold the working leg into a développé, keeping the back straight and the knee straight\n\t* Transition: Bring the working leg back to first position\n5. **Cloche** (4 counts)\n\t* Feet: First position, with the working foot (right) brushing out to the front and then to the back\n\t* Arms: Still in first position\n\t* Body: Maintain the engaged core and lengthened spine\n\t* Movement: Brush the working foot out to the front and then to the back, keeping the knee straight and the toe pointed\n\t* Transition: Bring the working foot back to first position\n\n**Preparation (4 counts):**\n\nStand in first position, with feet together and arms relaxed by your sides. Take a deep breath in, engaging your core and lengthening your spine. As you exhale, slowly raise your arms to first position, keeping them relaxed and curved.\n\n**Ending Phrase (4 counts):**\n\nFinish the sequence in first position, with feet together and arms in first position. Take a deep breath in, and as you exhale, slowly lower your arms to your sides, keeping them relaxed. Take a final deep breath in, and as you exhale, bow your head slightly, keeping your core engaged and your spine lengthened.\n\n**Justification:**\n\nThis sequence is designed to focus on battement tendu, with an emphasis on exploring different directions and adding various movements. The sequence begins with tendus to the front and sides, establishing a strong foundation and sense of direction. The added movements, such as fondu, cou de pied, and développé, challenge the dancers to control their movements and transitions. The cloche movement adds a dynamic element, requiring the dancers to quickly change direction and maintain balance. The moderate tempo and 4/4 time signature provide a steady and lively rhythm, allowing the dancers to focus on their technique and movement quality.'
display(Markdown(response))

**Sequence: Battement Tendu with Directions and Added Movements**

**Music:** A moderate tempo with a 4/4 time signature, ideally a classical piano piece with a steady and lively rhythm. (e.g., Debussy's "Clair de Lune")

**Tempo:** Moderate, around 100-104 beats per minute (BPM)

**Sequence:**

**Part 1: Tendus to the Front and Sides**

1. **Battement Tendu to the Front** (2 counts)
	* Feet: First position, with the working foot (right) brushing out to the front
	* Arms: Relaxed, with a slight curve, in first position (palms facing downwards)
	* Body: Engaged core, lengthened spine, and relaxed shoulders
	* Movement: Brush the working foot out to the front, keeping the knee straight and the toe pointed
	* Transition: Bring the working foot back to first position
2. **Battement Tendu to the Side (Right)** (2 counts)
	* Feet: First position, with the working foot (right) brushing out to the right side
	* Arms: Still in first position
	* Body: Maintain the engaged core and lengthened spine
	* Movement: Brush the working foot out to the right side, keeping the knee straight and the toe pointed
	* Transition: Bring the working foot back to first position
3. **Battement Tendu to the Side (Left)** (2 counts)
	* Feet: First position, with the working foot (left) brushing out to the left side
	* Arms: Still in first position
	* Body: Maintain the engaged core and lengthened spine
	* Movement: Brush the working foot out to the left side, keeping the knee straight and the toe pointed
	* Transition: Bring the working foot back to first position

**Part 2: Tendus to the Back and Added Movements**

1. **Battement Tendu to the Back** (2 counts)
	* Feet: First position, with the working foot (right) brushing out to the back
	* Arms: Still in first position
	* Body: Maintain the engaged core and lengthened spine
	* Movement: Brush the working foot out to the back, keeping the knee straight and the toe pointed
	* Transition: Bring the working foot back to first position
2. **Fondu** (2 counts)
	* Feet: First position, with the working foot (right) sinking into a fondu
	* Arms: Still in first position
	* Body: Maintain the engaged core and lengthened spine
	* Movement: Slowly lower the working foot into a fondu, keeping the back straight and the knee bent
	* Transition: Straighten the working leg and rise to the starting position
3. **Cou de Pied** (2 counts)
	* Feet: First position, with the working foot (right) drawing to the supporting leg
	* Arms: Still in first position
	* Body: Maintain the engaged core and lengthened spine
	* Movement: Draw the working foot to the supporting leg, keeping the knee bent and the toe pointed
	* Transition: Release the working foot and bring it back to first position
4. **Développé** (4 counts)
	* Feet: First position, with the working foot (right) unfolding into a développé
	* Arms: Still in first position
	* Body: Maintain the engaged core and lengthened spine
	* Movement: Slowly unfold the working leg into a développé, keeping the back straight and the knee straight
	* Transition: Bring the working leg back to first position
5. **Cloche** (4 counts)
	* Feet: First position, with the working foot (right) brushing out to the front and then to the back
	* Arms: Still in first position
	* Body: Maintain the engaged core and lengthened spine
	* Movement: Brush the working foot out to the front and then to the back, keeping the knee straight and the toe pointed
	* Transition: Bring the working foot back to first position

**Preparation (4 counts):**

Stand in first position, with feet together and arms relaxed by your sides. Take a deep breath in, engaging your core and lengthening your spine. As you exhale, slowly raise your arms to first position, keeping them relaxed and curved.

**Ending Phrase (4 counts):**

Finish the sequence in first position, with feet together and arms in first position. Take a deep breath in, and as you exhale, slowly lower your arms to your sides, keeping them relaxed. Take a final deep breath in, and as you exhale, bow your head slightly, keeping your core engaged and your spine lengthened.

**Justification:**

This sequence is designed to focus on battement tendu, with an emphasis on exploring different directions and adding various movements. The sequence begins with tendus to the front and sides, establishing a strong foundation and sense of direction. The added movements, such as fondu, cou de pied, and développé, challenge the dancers to control their movements and transitions. The cloche movement adds a dynamic element, requiring the dancers to quickly change direction and maintain balance. The moderate tempo and 4/4 time signature provide a steady and lively rhythm, allowing the dancers to focus on their technique and movement quality.

In [ ]:
response = conv.generate("Now you will design a sequence for rond de jambe.\
                         You should not focus on doing all  movements in all positions but,\
                         to make rond de jambe to all directions\
                         These are must haves:\
                         a) Petit battement sur le coud de pied \
                         b) Retiré/Passé \
                         c) Attitude avec plié")
display(Markdown(response))

In [13]:
response =  "**Sequence: Rond de Jambe with Directions and Added Movements**\n\n**Music:** A moderate tempo with a 3/4 time signature, ideally a classical piano piece with a steady and lilting rhythm. (e.g., Chopin's Waltz in D-flat major, Op. 64, No. 1)\n\n**Tempo:** Moderate, around 108-112 beats per minute (BPM)\n\n**Sequence:**\n\n**Part 1: Rond de Jambe to the Front and Sides**\n\n1. **Rond de Jambe à Terre to the Front** (4 counts)\n\t* Feet: First position, with the working foot (right) making small circles on the floor\n\t* Arms: Relaxed, with a slight curve, in first position (palms facing downwards)\n\t* Body: Engaged core, lengthened spine, and relaxed shoulders\n\t* Movement: Make small circles with the working foot, starting from the front and moving clockwise\n\t* Transition: Bring the working foot back to first position\n2. **Rond de Jambe à Terre to the Side (Right)** (4 counts)\n\t* Feet: First position, with the working foot (right) making small circles on the floor to the right side\n\t* Arms: Still in first position\n\t* Body: Maintain the engaged core and lengthened spine\n\t* Movement: Make small circles with the working foot, starting from the right side and moving clockwise\n\t* Transition: Bring the working foot back to first position\n3. **Rond de Jambe à Terre to the Side (Left)** (4 counts)\n\t* Feet: First position, with the working foot (left) making small circles on the floor to the left side\n\t* Arms: Still in first position\n\t* Body: Maintain the engaged core and lengthened spine\n\t* Movement: Make small circles with the working foot, starting from the left side and moving clockwise\n\t* Transition: Bring the working foot back to first position\n\n**Part 2: Rond de Jambe to the Back and Added Movements**\n\n1. **Rond de Jambe en l'Air to the Back** (4 counts)\n\t* Feet: First position, with the working foot (right) making small circles in the air to the back\n\t* Arms: Still in first position\n\t* Body: Maintain the engaged core and lengthened spine\n\t* Movement: Make small circles with the working foot, starting from the back and moving clockwise\n\t* Transition: Bring the working foot back to first position\n2. **Petit Battement sur le Coud de Pied** (2 counts)\n\t* Feet: First position, with the working foot (right) brushing against the supporting leg\n\t* Arms: Still in first position\n\t* Body: Maintain the engaged core and lengthened spine\n\t* Movement: Brush the working foot against the supporting leg, keeping the knee straight and the toe pointed\n\t* Transition: Release the working foot and bring it back to first position\n3. **Retiré/Passé** (2 counts)\n\t* Feet: First position, with the working foot (right) drawing to the supporting leg\n\t* Arms: Still in first position\n\t* Body: Maintain the engaged core and lengthened spine\n\t* Movement: Draw the working foot to the supporting leg, keeping the knee bent and the toe pointed\n\t* Transition: Release the working foot and bring it back to first position\n4. **Attitude avec Plié** (4 counts)\n\t* Feet: First position, with the working foot (right) lifting into attitude\n\t* Arms: Still in first position\n\t* Body: Maintain the engaged core and lengthened spine\n\t* Movement: Lift the working leg into attitude, keeping the back straight and the knee bent\n\t* Transition: Lower the working leg back to first position, pliéing the supporting leg\n\n**Preparation (4 counts):**\n\nStand in first position, with feet together and arms relaxed by your sides. Take a deep breath in, engaging your core and lengthening your spine. As you exhale, slowly raise your arms to first position, keeping them relaxed and curved.\n\n**Ending Phrase (4 counts):**\n\nFinish the sequence in first position, with feet together and arms in first position. Take a deep breath in, and as you exhale, slowly lower your arms to your sides, keeping them relaxed. Take a final deep breath in, and as you exhale, bow your head slightly, keeping your core engaged and your spine lengthened.\n\n**Justification:**\n\nThis sequence is designed to focus on rond de jambe, with an emphasis on exploring different directions and adding various movements. The sequence begins with rond de jambe à terre to the front and sides, establishing a strong foundation and sense of direction. The added movements, such as petit battement sur le coud de pied, retiré/passé, and attitude avec plié, challenge the dancers to control their movements and transitions. The moderate tempo and 3/4 time signature provide a steady and lilting rhythm, allowing the dancers to focus on their technique and movement quality."
display(Markdown(response))

**Sequence: Rond de Jambe with Directions and Added Movements**

**Music:** A moderate tempo with a 3/4 time signature, ideally a classical piano piece with a steady and lilting rhythm. (e.g., Chopin's Waltz in D-flat major, Op. 64, No. 1)

**Tempo:** Moderate, around 108-112 beats per minute (BPM)

**Sequence:**

**Part 1: Rond de Jambe to the Front and Sides**

1. **Rond de Jambe à Terre to the Front** (4 counts)
	* Feet: First position, with the working foot (right) making small circles on the floor
	* Arms: Relaxed, with a slight curve, in first position (palms facing downwards)
	* Body: Engaged core, lengthened spine, and relaxed shoulders
	* Movement: Make small circles with the working foot, starting from the front and moving clockwise
	* Transition: Bring the working foot back to first position
2. **Rond de Jambe à Terre to the Side (Right)** (4 counts)
	* Feet: First position, with the working foot (right) making small circles on the floor to the right side
	* Arms: Still in first position
	* Body: Maintain the engaged core and lengthened spine
	* Movement: Make small circles with the working foot, starting from the right side and moving clockwise
	* Transition: Bring the working foot back to first position
3. **Rond de Jambe à Terre to the Side (Left)** (4 counts)
	* Feet: First position, with the working foot (left) making small circles on the floor to the left side
	* Arms: Still in first position
	* Body: Maintain the engaged core and lengthened spine
	* Movement: Make small circles with the working foot, starting from the left side and moving clockwise
	* Transition: Bring the working foot back to first position

**Part 2: Rond de Jambe to the Back and Added Movements**

1. **Rond de Jambe en l'Air to the Back** (4 counts)
	* Feet: First position, with the working foot (right) making small circles in the air to the back
	* Arms: Still in first position
	* Body: Maintain the engaged core and lengthened spine
	* Movement: Make small circles with the working foot, starting from the back and moving clockwise
	* Transition: Bring the working foot back to first position
2. **Petit Battement sur le Coud de Pied** (2 counts)
	* Feet: First position, with the working foot (right) brushing against the supporting leg
	* Arms: Still in first position
	* Body: Maintain the engaged core and lengthened spine
	* Movement: Brush the working foot against the supporting leg, keeping the knee straight and the toe pointed
	* Transition: Release the working foot and bring it back to first position
3. **Retiré/Passé** (2 counts)
	* Feet: First position, with the working foot (right) drawing to the supporting leg
	* Arms: Still in first position
	* Body: Maintain the engaged core and lengthened spine
	* Movement: Draw the working foot to the supporting leg, keeping the knee bent and the toe pointed
	* Transition: Release the working foot and bring it back to first position
4. **Attitude avec Plié** (4 counts)
	* Feet: First position, with the working foot (right) lifting into attitude
	* Arms: Still in first position
	* Body: Maintain the engaged core and lengthened spine
	* Movement: Lift the working leg into attitude, keeping the back straight and the knee bent
	* Transition: Lower the working leg back to first position, pliéing the supporting leg

**Preparation (4 counts):**

Stand in first position, with feet together and arms relaxed by your sides. Take a deep breath in, engaging your core and lengthening your spine. As you exhale, slowly raise your arms to first position, keeping them relaxed and curved.

**Ending Phrase (4 counts):**

Finish the sequence in first position, with feet together and arms in first position. Take a deep breath in, and as you exhale, slowly lower your arms to your sides, keeping them relaxed. Take a final deep breath in, and as you exhale, bow your head slightly, keeping your core engaged and your spine lengthened.

**Justification:**

This sequence is designed to focus on rond de jambe, with an emphasis on exploring different directions and adding various movements. The sequence begins with rond de jambe à terre to the front and sides, establishing a strong foundation and sense of direction. The added movements, such as petit battement sur le coud de pied, retiré/passé, and attitude avec plié, challenge the dancers to control their movements and transitions. The moderate tempo and 3/4 time signature provide a steady and lilting rhythm, allowing the dancers to focus on their technique and movement quality.

In [ ]:
response = conv.generate("Now you will design a sequence for grand battement.\
                         You should not focus on doing all  movements in all positions but,\
                         to make them towards all directions\
                         Be detailed explaining each movement\
                         These are must haves:\
                         a) Arabesques \
                         b) Balances ")
display(Markdown(response))

In [14]:
response = "**Sequence: Grand Battement with Directions and Added Movements**\n\n**Music:** A moderate to fast tempo with a 4/4 time signature, ideally a classical piano piece with a strong and dramatic rhythm. (e.g., Tchaikovsky's Piano Concerto No. 1)\n\n**Tempo:** Moderate to fast, around 120-124 beats per minute (BPM)\n\n**Sequence:**\n\n**Part 1: Grand Battement to the Front and Sides**\n\n1. **Grand Battement to the Front** (4 counts)\n\t* Feet: First position, with the working foot (right) brushing out to the front\n\t* Arms: Relaxed, with a slight curve, in first position (palms facing downwards)\n\t* Body: Engaged core, lengthened spine, and relaxed shoulders\n\t* Movement: Brush the working foot out to the front, keeping the knee straight and the toe pointed\n\t* Transition: Bring the working foot back to first position\n\t* Note: Focus on using the core and leg muscles to generate power and control for the grand battement.\n2. **Grand Battement to the Side (Right)** (4 counts)\n\t* Feet: First position, with the working foot (right) brushing out to the right side\n\t* Arms: Still in first position\n\t* Body: Maintain the engaged core and lengthened spine\n\t* Movement: Brush the working foot out to the right side, keeping the knee straight and the toe pointed\n\t* Transition: Bring the working foot back to first position\n\t* Note: Keep the working foot in line with the supporting leg, and avoid letting it drift forward or backward.\n3. **Grand Battement to the Side (Left)** (4 counts)\n\t* Feet: First position, with the working foot (left) brushing out to the left side\n\t* Arms: Still in first position\n\t* Body: Maintain the engaged core and lengthened spine\n\t* Movement: Brush the working foot out to the left side, keeping the knee straight and the toe pointed\n\t* Transition: Bring the working foot back to first position\n\t* Note: Keep the working foot in line with the supporting leg, and avoid letting it drift forward or backward.\n\n**Part 2: Grand Battement to the Back and Added Movements**\n\n1. **Grand Battement to the Back** (4 counts)\n\t* Feet: First position, with the working foot (right) brushing out to the back\n\t* Arms: Still in first position\n\t* Body: Maintain the engaged core and lengthened spine\n\t* Movement: Brush the working foot out to the back, keeping the knee straight and the toe pointed\n\t* Transition: Bring the working foot back to first position\n\t* Note: Focus on using the core and leg muscles to generate power and control for the grand battement.\n2. **Arabesque (Right)** (4 counts)\n\t* Feet: First position, with the working foot (right) lifting into arabesque\n\t* Arms: Extended, with the right arm reaching forward and the left arm reaching backward\n\t* Body: Maintain the engaged core and lengthened spine\n\t* Movement: Lift the working leg into arabesque, keeping the back straight and the knee straight\n\t* Transition: Lower the working leg back to first position\n\t* Note: Focus on maintaining balance and control throughout the arabesque.\n3. **Arabesque (Left)** (4 counts)\n\t* Feet: First position, with the working foot (left) lifting into arabesque\n\t* Arms: Extended, with the left arm reaching forward and the right arm reaching backward\n\t* Body: Maintain the engaged core and lengthened spine\n\t* Movement: Lift the working leg into arabesque, keeping the back straight and the knee straight\n\t* Transition: Lower the working leg back to first position\n\t* Note: Focus on maintaining balance and control throughout the arabesque.\n4. **Balance (Right)** (4 counts)\n\t* Feet: First position, with the working foot (right) lifting into balance\n\t* Arms: Extended, with the right arm reaching forward and the left arm reaching backward\n\t* Body: Maintain the engaged core and lengthened spine\n\t* Movement: Lift the working leg into balance, keeping the back straight and the knee bent\n\t* Transition: Lower the working leg back to first position\n\t* Note: Focus on maintaining balance and control throughout the balance.\n5. **Balance (Left)** (4 counts)\n\t* Feet: First position, with the working foot (left) lifting into balance\n\t* Arms: Extended, with the left arm reaching forward and the right arm reaching backward\n\t* Body: Maintain the engaged core and lengthened spine\n\t* Movement: Lift the working leg into balance, keeping the back straight and the knee bent\n\t* Transition: Lower the working leg back to first position\n\t* Note: Focus on maintaining balance and control throughout the balance.\n\n**Preparation (4 counts):**\n\nStand in first position, with feet together and arms relaxed by your sides. Take a deep breath in, engaging your core and lengthening your spine. As you exhale, slowly raise your arms to first position, keeping them relaxed and curved.\n\n**Ending Phrase (4 counts):**\n\nFinish the sequence in first position, with feet together and arms in first position. Take a deep breath in, and as you exhale, slowly lower your arms to your sides, keeping them relaxed. Take a final deep breath in, and as you exhale, bow your head slightly, keeping your core engaged and your spine lengthened.\n\n**Justification:**\n\nThis sequence is designed to focus on grand battement, with an emphasis on exploring different directions and adding various movements. The sequence begins with grand battement to the front and sides, establishing a strong foundation and sense of direction. The added movements, such as arabesque and balance, challenge the dancers to control their movements and transitions. The moderate to fast tempo and 4/4 time signature provide a strong and dramatic rhythm, allowing the dancers to focus on their technique and movement quality."
display(Markdown(response))

**Sequence: Grand Battement with Directions and Added Movements**

**Music:** A moderate to fast tempo with a 4/4 time signature, ideally a classical piano piece with a strong and dramatic rhythm. (e.g., Tchaikovsky's Piano Concerto No. 1)

**Tempo:** Moderate to fast, around 120-124 beats per minute (BPM)

**Sequence:**

**Part 1: Grand Battement to the Front and Sides**

1. **Grand Battement to the Front** (4 counts)
	* Feet: First position, with the working foot (right) brushing out to the front
	* Arms: Relaxed, with a slight curve, in first position (palms facing downwards)
	* Body: Engaged core, lengthened spine, and relaxed shoulders
	* Movement: Brush the working foot out to the front, keeping the knee straight and the toe pointed
	* Transition: Bring the working foot back to first position
	* Note: Focus on using the core and leg muscles to generate power and control for the grand battement.
2. **Grand Battement to the Side (Right)** (4 counts)
	* Feet: First position, with the working foot (right) brushing out to the right side
	* Arms: Still in first position
	* Body: Maintain the engaged core and lengthened spine
	* Movement: Brush the working foot out to the right side, keeping the knee straight and the toe pointed
	* Transition: Bring the working foot back to first position
	* Note: Keep the working foot in line with the supporting leg, and avoid letting it drift forward or backward.
3. **Grand Battement to the Side (Left)** (4 counts)
	* Feet: First position, with the working foot (left) brushing out to the left side
	* Arms: Still in first position
	* Body: Maintain the engaged core and lengthened spine
	* Movement: Brush the working foot out to the left side, keeping the knee straight and the toe pointed
	* Transition: Bring the working foot back to first position
	* Note: Keep the working foot in line with the supporting leg, and avoid letting it drift forward or backward.

**Part 2: Grand Battement to the Back and Added Movements**

1. **Grand Battement to the Back** (4 counts)
	* Feet: First position, with the working foot (right) brushing out to the back
	* Arms: Still in first position
	* Body: Maintain the engaged core and lengthened spine
	* Movement: Brush the working foot out to the back, keeping the knee straight and the toe pointed
	* Transition: Bring the working foot back to first position
	* Note: Focus on using the core and leg muscles to generate power and control for the grand battement.
2. **Arabesque (Right)** (4 counts)
	* Feet: First position, with the working foot (right) lifting into arabesque
	* Arms: Extended, with the right arm reaching forward and the left arm reaching backward
	* Body: Maintain the engaged core and lengthened spine
	* Movement: Lift the working leg into arabesque, keeping the back straight and the knee straight
	* Transition: Lower the working leg back to first position
	* Note: Focus on maintaining balance and control throughout the arabesque.
3. **Arabesque (Left)** (4 counts)
	* Feet: First position, with the working foot (left) lifting into arabesque
	* Arms: Extended, with the left arm reaching forward and the right arm reaching backward
	* Body: Maintain the engaged core and lengthened spine
	* Movement: Lift the working leg into arabesque, keeping the back straight and the knee straight
	* Transition: Lower the working leg back to first position
	* Note: Focus on maintaining balance and control throughout the arabesque.
4. **Balance (Right)** (4 counts)
	* Feet: First position, with the working foot (right) lifting into balance
	* Arms: Extended, with the right arm reaching forward and the left arm reaching backward
	* Body: Maintain the engaged core and lengthened spine
	* Movement: Lift the working leg into balance, keeping the back straight and the knee bent
	* Transition: Lower the working leg back to first position
	* Note: Focus on maintaining balance and control throughout the balance.
5. **Balance (Left)** (4 counts)
	* Feet: First position, with the working foot (left) lifting into balance
	* Arms: Extended, with the left arm reaching forward and the right arm reaching backward
	* Body: Maintain the engaged core and lengthened spine
	* Movement: Lift the working leg into balance, keeping the back straight and the knee bent
	* Transition: Lower the working leg back to first position
	* Note: Focus on maintaining balance and control throughout the balance.

**Preparation (4 counts):**

Stand in first position, with feet together and arms relaxed by your sides. Take a deep breath in, engaging your core and lengthening your spine. As you exhale, slowly raise your arms to first position, keeping them relaxed and curved.

**Ending Phrase (4 counts):**

Finish the sequence in first position, with feet together and arms in first position. Take a deep breath in, and as you exhale, slowly lower your arms to your sides, keeping them relaxed. Take a final deep breath in, and as you exhale, bow your head slightly, keeping your core engaged and your spine lengthened.

**Justification:**

This sequence is designed to focus on grand battement, with an emphasis on exploring different directions and adding various movements. The sequence begins with grand battement to the front and sides, establishing a strong foundation and sense of direction. The added movements, such as arabesque and balance, challenge the dancers to control their movements and transitions. The moderate to fast tempo and 4/4 time signature provide a strong and dramatic rhythm, allowing the dancers to focus on their technique and movement quality.

In [ ]:
response = conv.generate("Now you will design a sequence for streteching.\
                         Propose some barre and some floor stretching movements")
display(Markdown(response))

In [15]:
response = "**Sequence: Stretching**\n\n**Music:** A slow and calming tempo with a 4/4 time signature, ideally a classical piano piece or nature sounds. (e.g., Chopin's Nocturne in E-flat major, Op. 9, No. 2)\n\n**Tempo:** Slow, around 60-64 beats per minute (BPM)\n\n**Barre Stretching (10 counts):**\n\n1. **Plié in First Position** (2 counts)\n\t* Feet: Heels together, toes turned out\n\t* Arms: Relaxed, with a slight curve, in first position (palms facing downwards)\n\t* Body: Engaged core, lengthened spine, and relaxed shoulders\n\t* Movement: Slowly lower the body down into a plié, keeping the back straight and the knees over the toes\n2. **Tendu to the Front** (2 counts)\n\t* Feet: First position, with the working foot (right) brushing out to the front\n\t* Arms: Still in first position\n\t* Body: Maintain the engaged core and lengthened spine\n\t* Movement: Brush the working foot out to the front, keeping the knee straight and the toe pointed\n3. **Stretch to the Front** (2 counts)\n\t* Feet: First position, with the working foot (right) stretching out to the front\n\t* Arms: Extended, with the right arm reaching forward and the left arm reaching backward\n\t* Body: Maintain the engaged core and lengthened spine\n\t* Movement: Stretch the working leg out to the front, keeping the back straight and the knee straight\n4. **Plié in First Position** (2 counts)\n\t* Feet: Heels together, toes turned out\n\t* Arms: Relaxed, with a slight curve, in first position (palms facing downwards)\n\t* Body: Engaged core, lengthened spine, and relaxed shoulders\n\t* Movement: Slowly lower the body down into a plié, keeping the back straight and the knees over the toes\n5. **Tendu to the Side** (2 counts)\n\t* Feet: First position, with the working foot (right) brushing out to the side\n\t* Arms: Still in first position\n\t* Body: Maintain the engaged core and lengthened spine\n\t* Movement: Brush the working foot out to the side, keeping the knee straight and the toe pointed\n\n**Floor Stretching (16 counts):**\n\n1. **Hamstring Stretch** (4 counts)\n\t* Feet: Sitting on the floor with the legs straight out in front\n\t* Arms: Extended, with the hands reaching forward\n\t* Body: Engaged core, lengthened spine, and relaxed shoulders\n\t* Movement: Lean forward, stretching the hamstrings and keeping the back straight\n2. **Hip Flexor Stretch** (4 counts)\n\t* Feet: Kneeling on the floor with the right knee bent and the left leg straight out behind\n\t* Arms: Extended, with the right arm reaching forward and the left arm reaching backward\n\t* Body: Engaged core, lengthened spine, and relaxed shoulders\n\t* Movement: Lean forward, stretching the hip flexor and keeping the back straight\n3. **Quad Stretch** (4 counts)\n\t* Feet: Standing on the floor with the right leg bent behind and the left leg straight out in front\n\t* Arms: Extended, with the right arm reaching backward and the left arm reaching forward\n\t* Body: Engaged core, lengthened spine, and relaxed shoulders\n\t* Movement: Lean forward, stretching the quadriceps and keeping the back straight\n4. **Calf Stretch** (4 counts)\n\t* Feet: Standing on the floor with the right leg straight out in front and the left leg bent behind\n\t* Arms: Extended, with the right arm reaching forward and the left arm reaching backward\n\t* Body: Engaged core, lengthened spine, and relaxed shoulders\n\t* Movement: Lean forward, stretching the calf muscles and keeping the back straight\n\n**Preparation (4 counts):**\n\nStand in first position, with feet together and arms relaxed by your sides. Take a deep breath in, engaging your core and lengthening your spine. As you exhale, slowly raise your arms to first position, keeping them relaxed and curved.\n\n**Ending Phrase (4 counts):**\n\nFinish the sequence in a relaxed position, with feet together and arms by your sides. Take a deep breath in, and as you exhale, slowly lower your arms to your sides, keeping them relaxed. Take a final deep breath in, and as you exhale, bow your head slightly, keeping your core engaged and your spine lengthened.\n\n**Justification:**\n\nThis sequence is designed to focus on stretching, with an emphasis on releasing tension and improving flexibility. The barre stretching movements target the major muscle groups, including the hamstrings, quadriceps, and hip flexors. The floor stretching movements target the same muscle groups, as well as the calf muscles. The slow tempo and calming music provide a relaxing atmosphere, allowing the dancers to focus on their breathing and stretching."
display(Markdown(response))

**Sequence: Stretching**

**Music:** A slow and calming tempo with a 4/4 time signature, ideally a classical piano piece or nature sounds. (e.g., Chopin's Nocturne in E-flat major, Op. 9, No. 2)

**Tempo:** Slow, around 60-64 beats per minute (BPM)

**Barre Stretching (10 counts):**

1. **Plié in First Position** (2 counts)
	* Feet: Heels together, toes turned out
	* Arms: Relaxed, with a slight curve, in first position (palms facing downwards)
	* Body: Engaged core, lengthened spine, and relaxed shoulders
	* Movement: Slowly lower the body down into a plié, keeping the back straight and the knees over the toes
2. **Tendu to the Front** (2 counts)
	* Feet: First position, with the working foot (right) brushing out to the front
	* Arms: Still in first position
	* Body: Maintain the engaged core and lengthened spine
	* Movement: Brush the working foot out to the front, keeping the knee straight and the toe pointed
3. **Stretch to the Front** (2 counts)
	* Feet: First position, with the working foot (right) stretching out to the front
	* Arms: Extended, with the right arm reaching forward and the left arm reaching backward
	* Body: Maintain the engaged core and lengthened spine
	* Movement: Stretch the working leg out to the front, keeping the back straight and the knee straight
4. **Plié in First Position** (2 counts)
	* Feet: Heels together, toes turned out
	* Arms: Relaxed, with a slight curve, in first position (palms facing downwards)
	* Body: Engaged core, lengthened spine, and relaxed shoulders
	* Movement: Slowly lower the body down into a plié, keeping the back straight and the knees over the toes
5. **Tendu to the Side** (2 counts)
	* Feet: First position, with the working foot (right) brushing out to the side
	* Arms: Still in first position
	* Body: Maintain the engaged core and lengthened spine
	* Movement: Brush the working foot out to the side, keeping the knee straight and the toe pointed

**Floor Stretching (16 counts):**

1. **Hamstring Stretch** (4 counts)
	* Feet: Sitting on the floor with the legs straight out in front
	* Arms: Extended, with the hands reaching forward
	* Body: Engaged core, lengthened spine, and relaxed shoulders
	* Movement: Lean forward, stretching the hamstrings and keeping the back straight
2. **Hip Flexor Stretch** (4 counts)
	* Feet: Kneeling on the floor with the right knee bent and the left leg straight out behind
	* Arms: Extended, with the right arm reaching forward and the left arm reaching backward
	* Body: Engaged core, lengthened spine, and relaxed shoulders
	* Movement: Lean forward, stretching the hip flexor and keeping the back straight
3. **Quad Stretch** (4 counts)
	* Feet: Standing on the floor with the right leg bent behind and the left leg straight out in front
	* Arms: Extended, with the right arm reaching backward and the left arm reaching forward
	* Body: Engaged core, lengthened spine, and relaxed shoulders
	* Movement: Lean forward, stretching the quadriceps and keeping the back straight
4. **Calf Stretch** (4 counts)
	* Feet: Standing on the floor with the right leg straight out in front and the left leg bent behind
	* Arms: Extended, with the right arm reaching forward and the left arm reaching backward
	* Body: Engaged core, lengthened spine, and relaxed shoulders
	* Movement: Lean forward, stretching the calf muscles and keeping the back straight

**Preparation (4 counts):**

Stand in first position, with feet together and arms relaxed by your sides. Take a deep breath in, engaging your core and lengthening your spine. As you exhale, slowly raise your arms to first position, keeping them relaxed and curved.

**Ending Phrase (4 counts):**

Finish the sequence in a relaxed position, with feet together and arms by your sides. Take a deep breath in, and as you exhale, slowly lower your arms to your sides, keeping them relaxed. Take a final deep breath in, and as you exhale, bow your head slightly, keeping your core engaged and your spine lengthened.

**Justification:**

This sequence is designed to focus on stretching, with an emphasis on releasing tension and improving flexibility. The barre stretching movements target the major muscle groups, including the hamstrings, quadriceps, and hip flexors. The floor stretching movements target the same muscle groups, as well as the calf muscles. The slow tempo and calming music provide a relaxing atmosphere, allowing the dancers to focus on their breathing and stretching.